# TODO:
    - Refine narrative
    - Make plots
    - Determine presentation form
        - Kyle: partial to some kind of web app

## Notes

- DB- Note on file names.  I slightly edited file names (I think to include a -) so the names  would be sufficiently uniform, for reading into dfs.  Let's ensure this is pushed correctly. 
- RE: CSV file inputs and date ranges.  I observe there is no 2022 autonomous vehicle disengagement report- driverless on the DMV website
- Similarily, there is something going on in the driverless mileage reports.
- The 2020-21-Autonomous-Mileage-Reports-CSV-driverless covers Jan 2020 - Nov 2021
- The 2021-22-Autonomous-Mileage-Reports-CSV-driverless covers Jan 2021 - Nov 2022, but only has 3 lines, for WERIDE CORP,
    - Which doesn't appear in the 2020-21 entry.  This is probably due to a delayed filing on their part, and staff did the ez thing.
- The 2022 Report starts in Dec 2021 - Nov 2022, and doesn't include WERIDE info (either by name or permit #)
- The 2023 Report covers Dec 2022- Nov 2023, and includes WERIDE, so there is no overlaping info.

- Re: The mileage reports.  I'm loading them in so the yeoman's work is done.  I don't want to botch joining them.
   - Some form of groupby + set_index, as per discord, might be the trick here. 
   - Alternatively, we can combine the permit # + VIN into a new single column, and this will enable some straightforward y/y granular tracking via joined dfs (m/m from Jan 2020 - Nov 2023)
- Finally, we should be on the lookout for permit #s that have company name changes, like the company got bought or something

## Narrative, high level, keep refining into story:
1. Overall trends, total miles, leaders in the space, etc.
2. Cruze story because it's interesting (backup: Waymo)
3. Follow 'zippy' the car (longest hauler)

# IMPORTS

In [ ]:
import pandas as pd
import seaborn as sns 
import numpy
import plotly.express as px
from pprint import pprint
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS


# DATA

In [ ]:
### Lists
## Years common across data
years = ["2020", "2021", "2022", "2023"]
years = sorted(list(years))

months = ["DEC-1", "JAN", "FEB", "MAR", "APR", "MAY", "JUN", "JULY", "AUG", "SEP", "OCT", "NOV"]


### Mileage

In [ ]:
# TODO: Cast DATE column as datetype for disengagements

## Read in mileage data
mile_dfs = []
for year in years:
    try:
        annual_mileage = pd.read_csv(f"data/Mileage/{year}-Autonomous-Mileage-Reports-CSV.csv", encoding='utf-8')
    # Allows the search to be broad
    except FileNotFoundError:
        print(f"File for year {year} not found. Skipping...")
        continue
    # Got some kind of unicode error. This fixed it.
    except UnicodeDecodeError:
        # If utf-8 decoding fails, try another common encoding
        annual_mileage = pd.read_csv(f"data/Mileage/{year}-Autonomous-Mileage-Reports-CSV.csv", encoding='latin-1')
    annual_mileage["Year"] = year
    annual_mileage.rename(columns={'ANNUAL TOTAL': 'Mileage Total'}, inplace=True)
    for i, month in enumerate(months, start=4):
        annual_mileage.columns.values[i] = month
    mile_dfs.append(annual_mileage)
    # INFO: each annual report has 22 cols

mileage_df = pd.concat(mile_dfs, ignore_index=True)
# mileage_df.set_index('VIN NUMBER', inplace=True)

First Time Filers - Mileage

In [ ]:
mile_first_time_filer_dfs = []
for year in years:
    try:
        annual_disengagements = pd.read_csv(f"data/Mileage/{year}-{str(int(year[3]) + 21)}-Autonomous-Mileage-Reports-CSV-first-time-filers.csv", encoding='utf-8')
    except FileNotFoundError:
        print(f"File for year {year}-{str(int(year[3]) + 21)} not found. Skipping...")
        continue
    # Got some kind of unicode error. This fixed it.
    except UnicodeDecodeError:
        # If utf-8 decoding fails, try another common encoding
        annual_disengagements = pd.read_csv(f"data/Mileage/{year}-{str(int(year[3]) + 21)}-Autonomous-Mileage-Reports-CSV-first-time-filers.csv", encoding='latin-1')
    mile_first_time_filer_dfs.append(annual_disengagements)
    
# Not combining due to join issue raised at top of document
#first_time_mileage_df = pd.concat(mile_first_time_filer, ignore_index=True)

#print(mile_first_time_filer_dfs[0].head())
print("# of objects in mile_driverless_dfs:  " + str(len(mile_first_time_filer_dfs)))

Driverless Mileage

In [ ]:
# DB - Using the same process I used above, for Autonomous-Mileage-Reports-CSV-driverless.csv & Autonomous-Mileage-Reports-CSV-first-time-filers.csv
mile_driverless_dfs = []
for year in years:
    try:
        annual_disengagements = pd.read_csv(f"data/Mileage/{year}-{str(int(year[3]) + 21)}-Autonomous-Mileage-Reports-CSV-driverless.csv", encoding='utf-8')
    except FileNotFoundError:
        print(f"File for year {year}-{str(int(year[3]) + 21)} not found. Skipping...")
        continue
    # Got some kind of unicode error. This fixed it.
    except UnicodeDecodeError:
        # If utf-8 decoding fails, try another common encoding
        annual_disengagements = pd.read_csv(f"data/Mileage/{year}-{str(int(year[3]) + 21)}-Autonomous-Mileage-Reports-CSV-driverless.csv", encoding='latin-1')
    annual_disengagements["Year"] = year
    for i, month in enumerate(months, start=4):
        annual_disengagements.columns.values[i] = month
    mile_driverless_dfs.append(annual_disengagements)

for year in years:    
    try:
        annual_disengagements = pd.read_csv(f"data/Mileage/{year}-Autonomous-Mileage-Reports-CSV-driverless.csv", encoding='utf-8')
        # Allows the search to be broad
    except FileNotFoundError:
        print(f"File for year {year} not found. Skipping...")
        continue
        # Got some kind of unicode error. This fixed it.
    except UnicodeDecodeError:
        # If utf-8 decoding fails, try another common encoding
        annual_disengagements = pd.read_csv(f"data/Mileage/{year}-Autonomous-Mileage-Reports-CSV-driverless.csv", encoding='latin-1')
    annual_disengagements["Year"] = year
    for i, month in enumerate(months, start=4):
        annual_disengagements.columns.values[i] = month
    mile_driverless_dfs.append(annual_disengagements)

driverless_mileage_df = pd.concat(mile_driverless_dfs, ignore_index=True)
# TODO: deal with stupid double year file.

#print(mile_driverless_dfs[0].head())
print("# of objects in mile_driverless_dfs:  " + str(len(mile_driverless_dfs)))
driverless_mileage_df.head()


Combined Mileage

In [ ]:
# TODO Duplicate and modify code from combined disengagements here


### Disengagements

In [ ]:
### Read in data
## Read in disengagements data
dis_dfs = []
for year in years:
    try:
        annual_disengagements = pd.read_csv(f"data/Disengagements/{year}-Autonomous-Vehicle-Disengagement-Reports-CSV.csv", encoding='utf-8')
    # Allows the search to be broad
    except FileNotFoundError:
        print(f"File for year {year} not found. Skipping...")
        continue
    # Got some kind of unicode error. This fixed it.
    except UnicodeDecodeError:
        # If utf-8 decoding fails, try another common encoding
        annual_disengagements = pd.read_csv(f"data/Disengagements/{year}-Autonomous-Vehicle-Disengagement-Reports-CSV.csv", encoding='latin-1')
    dis_dfs.append(annual_disengagements)
    
disengagement_df = pd.concat(dis_dfs, ignore_index=True)

#DB.  The unnamed column 10 only appears in the 2021 report, and the 71 entries are all for IMAGRY INC.,
# Reading "Perception" or "Planning"
# Dropped Column
disengagement_df = disengagement_df.drop('Unnamed: 9', axis=1)
#Apx 8k  empty rows.  Droped all rows with any nulls.  
disengagement_df.dropna(axis=0, how='any', inplace=True)

First Time Filers - Disengagements

In [ ]:
# Adding in 1st time filer Disengagements.  Utilizing Kyle's code from above
first_time_dis_dfs = []
for year in years:
    try:
        annual_disengagements = pd.read_csv(f"data/Disengagements/{year}-{str(int(year[3]) + 21)}-Autonomous-Vehicle-Disengagement-Reports-CSV-first-time-filers.csv", encoding='utf-8')
    except FileNotFoundError:
        print(f"File for year {year}-{str(int(year[3]) + 21)} not found. Skipping...")
        continue
    # Got some kind of unicode error. This fixed it.
    except UnicodeDecodeError:
        # If utf-8 decoding fails, try another common encoding
        annual_disengagements = pd.read_csv(f"data/Disengagements/{year}-{str(int(year[3]) + 21)}-Autonomous-Vehicle-Disengagement-Reports-CSV-first-time-filers.csv", encoding='latin-1')
    first_time_dis_dfs.append(annual_disengagements)
    
first_df = pd.concat(first_time_dis_dfs, ignore_index=True)
#Some 200ish null rows imported. Dropped any row with any missing data
first_df.dropna(axis=0, how='any', inplace=True)

Driverless Disengagements

In [ ]:
# Driverless reports.  2022 is missing.  There are only 2 files here.  I've combined the above 2x for loops to snag them both, 
# since they have different naming conventions
driverless_dis_dfs = []
for year in years:
    try:
        annual_disengagements = pd.read_csv(f"data/Disengagements/{year}-{str(int(year[3]) + 21)}-Autonomous-Vehicle-Disengagement-Reports-CSV-driverless.csv", encoding='utf-8')
    except FileNotFoundError:
        print(f"File for year {year}-{str(int(year[3]) + 21)} not found. Skipping...")
        continue
    # Got some kind of unicode error. This fixed it.
    except UnicodeDecodeError:
        # If utf-8 decoding fails, try another common encoding
        annual_disengagements = pd.read_csv(f"data/Disengagements/{year}-{str(int(year[3]) + 21)}-Autonomous-Vehicle-Disengagement-Reports-CSV-driverless.csv", encoding='latin-1')
    driverless_dis_dfs.append(annual_disengagements)

for year in years:    
    try:
        annual_disengagements = pd.read_csv(f"data/Disengagements/{year}-Autonomous-Vehicle-Disengagement-Reports-CSV-driverless.csv", encoding='utf-8')
        # Allows the search to be broad
    except FileNotFoundError:
        print(f"File for year {year} not found. Skipping...")
        continue
        # Got some kind of unicode error. This fixed it.
    except UnicodeDecodeError:
        # If utf-8 decoding fails, try another common encoding
        annual_disengagements = pd.read_csv(f"data/Disengagements/{year}-Autonomous-Vehicle-Disengagement-Reports-CSV-driverless.csv", encoding='latin-1')
    driverless_dis_dfs.append(annual_disengagements)

autonomous_dis_df = pd.concat(driverless_dis_dfs, ignore_index=True)
# Dropping some 2000+ null rows.
autonomous_dis_df.dropna(axis=0, how='any', inplace=True)

Combined Disengagements

In [ ]:
# 3x active DFs for the disengagement info are
# disengagement_df
# first_df
# autonomous_dis_df
# List of these objects to for loop the DATE cleanup.  
# IF we wanted to acccess the report source from the combined df, we could add a column to each, with the report source

# Initially had trouble by converting the dates to_datetime and then trying to drop the hour/min/sec.
# Resolved by first converting DATE to str, dropping everything after " " (since all times were separated from date by a space)
# And then subsequently converting the column to_datetime

# This works, but can't be re-run unless above code blocks that create the dfs are also re-run (I think).  
# Used print(i['DATE'].value_counts()) to confirm this came out correctly.  

disengagement_dfs_list = [disengagement_df, first_df, autonomous_dis_df]

for i in disengagement_dfs_list:
    i['DATE'] = i['DATE'].astype(str)
    i["DATE"] = i["DATE"].str.split(" ", expand=True)[0]
    i['DATE'] = pd.to_datetime(i['DATE'], errors='raise', dayfirst=True, format='mixed')

In [ ]:
'''
for i in disengagement_dfs_list:
    #print(i['Permit Number'].value_counts())
    print(i.shape)
'''

# Operator-in-Vehicle = AVT Permit prefix
# NVO = AVDT Permit Prefix.
# I'm fairly certain splitting these into distinct columns for purposes of multi-indexed rows or columns is straightforward, 
# So am moving forward with data cleaning

In [ ]:
# 3x disengagement df's combined together into one, single column for all permit info
combined_dis_df = pd.concat(disengagement_dfs_list, ignore_index=True)


# Data Cleaning and QA

Drop Columns

In [ ]:
# What are "Unnamed" colums? Probably useless "reserved"
# This and below are from kyle, and deal with the initial mileage_df
unnamed_cols = ['Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21']
for check in unnamed_cols:
    non_null_count = mileage_df[check].notna().sum()
    print(f'Number of non values in {check}: {non_null_count}')

# Empty. Drop 'em
mileage_df = mileage_df.drop(columns=unnamed_cols)
mileage_df.columns


Set Types

In [ ]:
### Mileage
print(f'Before: {mileage_df.dtypes}')
cast_as_float = [
       'Annual Total of Disengagements', 'DEC-1', 'JAN', 'FEB', 'MAR', 'APR',
       'MAY', 'JUN', 'JULY', 'AUG', 'SEP', 'OCT', 'NOV', 'Mileage Total',
       ]

for col in cast_as_float:
    mileage_df[col] = mileage_df[col].apply(lambda x: x.replace(',', '').replace(' ', '') if isinstance(x, str) else x)
    mileage_df[col] = mileage_df[col].astype(float)

# Assuming df is your DataFrame and 'Column_Name' is the name of the column you want to convert to represent years
mileage_df['Year'] = pd.to_datetime(mileage_df['Year']).dt.year
mileage_df["Manufacturer"] = mileage_df["Manufacturer"].astype(str)


print(f'After: {mileage_df.dtypes}')



QA: Ensure sum of monthly miles still equals Annual totals after column shuffling

In [ ]:
### QA the monthly vs sum values
# Calculate the sum of values in the month columns for each row
month_sums = mileage_df.iloc[:, 4:-2].sum(axis=1)  

# Calculate the absolute difference between the sum of month columns and the "Mileage Total" column
absolute_difference = abs(month_sums - mileage_df['Mileage Total'])

# Check if the absolute difference is less than or equal to 2.0
within_tolerance = absolute_difference <= 2.0
fishy = (~within_tolerance).sum()
print("Number of rows where months do not add up to within 1.0 of annual total:", fishy)

# Print rows where the absolute difference is not within 2.0
# print(mileage_df[~within_tolerance])
qa = mileage_df[~within_tolerance]
print(f'Rows where MileageTotal is null: {qa["Mileage Total"].isnull().sum()}')
qa.head()


# count all nulls 
print(mileage_df.isnull().sum())
mileage_df.dropna(subset=['Mileage Total'], inplace=True)
# Drops majority of other nulls
print(mileage_df.isnull().sum())


Rename Columns

In [ ]:
combined_dis_df.rename(columns = {"Manufacturer" : "OPERATOR",
                                  "DISENGAGEMENT\nLOCATION\n(Interstate, Freeway, Highway, Rural Road, Street, or Parking Facility)" : "DISENGAGEMENT LOCATION", 
                                  "DISENGAGEMENT INITIATED BY\n(AV System, Test Driver, Remote Operator, or Passenger)" : "INITIATED BY",
                                  "DRIVER PRESENT\n(Yes or No)" : "DRIVER PRESENT",
                                  "VEHICLE IS CAPABLE OF OPERATING WITHOUT A DRIVER\n(Yes or No)" : "NVO CAPABLE"}, inplace=True)


mileage_df.rename(columns = {"Manufacturer" : "OPERATOR"}, inplace=True)
combined_dis_df.head(10)



Reduce Duplicates

In [ ]:

# DEDUPLICATOR
def polish(df, column):
   df[column]=df[column].str.upper() # Capitalize everything
   df[column] = df[column].str.replace("[.,]", "", regex=True) # Remove periods and commas 
   df[column] = df[column].str.replace(r'\b(LLC|INC)\b', '', regex=True)  # Remove "LLC" and "INC"
   df[column]=df[column].str.rstrip() # Remove trailing whitespace

   return df 

# Print statements are for verification and team review. Might leave them to show class "This is the mess of data we had to clean up" - discuss
print(f'Locations: {combined_dis_df["DISENGAGEMENT LOCATION"].nunique()} - {sorted(combined_dis_df["DISENGAGEMENT LOCATION"].unique())}')
print(f'Operators: {combined_dis_df["OPERATOR"].nunique()} - {sorted(combined_dis_df["OPERATOR"].unique())}')
print()

dis_columns_to_clean = ["DISENGAGEMENT LOCATION", "OPERATOR"]

# Deuplicate Disengagements
for i in dis_columns_to_clean:
   combined_dis_df = polish(combined_dis_df, i)
# Set VIN as Index
#combined_dis_df.set_index("VIN NUMBER", inplace=True)


print(f'Cleaned locations: {combined_dis_df["DISENGAGEMENT LOCATION"].nunique()} - {sorted(combined_dis_df["DISENGAGEMENT LOCATION"].unique())}')
print(f'Cleaned operators: {combined_dis_df["OPERATOR"].nunique()} - {sorted(combined_dis_df["OPERATOR"].unique())}')
print()

# Deduplicate Mileage
print("Original number of operators: ", mileage_df["OPERATOR"].nunique())

miles_columns_to_clean = ["OPERATOR"]

# Deuplicate Mileage
for i in miles_columns_to_clean:
   mileage_df = polish(mileage_df, i)

print("Actual number of unique operators: ", mileage_df["OPERATOR"].nunique())

operators = sorted(mileage_df["OPERATOR"].unique())
for value in operators:
    print(value)

# TODO: Mop up the last operators name alignment
# mileage_df["Manufacturer"] = mileage_df["Manufacturer"].str.replace('long_duplicates', 'shorter_version')


### Find 'Zippy'

In [332]:
miles_by_vin = mileage_df.groupby("VIN NUMBER")["Mileage Total"].sum().sort_values(ascending=False)
miles_by_vin.head()
zippy = miles_by_vin.idxmax()
print(f'Zippy VIN is {zippy}')

print(f'Zippy is a {mileage_df.loc[mileage_df["VIN NUMBER"] == zippy, "OPERATOR"].unique()[0]} vehicle.')

Zippy VIN is SADHW2S13M1616427
Zippy is a WAYMO vehicle.


### GRAPHS

In [ ]:
nonzero_mileage = mileage_df[(mileage_df["Mileage Total"] > 0)]

In [ ]:
###  autonomous miles necessarily.  Need to look at data sources further.

# TODO: Drop zeroes? Probably.

#### Histogram: miles driven / VIN ######
vin = px.histogram(nonzero_mileage,
             x='Mileage Total', color="Manufacturer", title="Autonomous Mileage by Vehicle (2021-2023)", nbins=50)
vin.update_layout(xaxis_title='Vehicle Mileage', yaxis_title='Number of Vehicles')
vin.show()

#### Histogram: miles driven / Operator ######
manufacturer_miles_sum = nonzero_mileage.groupby('Manufacturer')['Mileage Total'].sum().reset_index()

fig = px.histogram(manufacturer_miles_sum, x='Mileage Total', title='Total Miles Driven Per Operator (2021-2023)',
                   labels={'Mileage Total': 'Total Miles Driven'},  nbins=100)  # Adjust nbins as needed
fig.update_layout(yaxis_title='Operators')

fig.show()




#### WANT: Would be absolutley sick if we could start with the VINs histogram and animated into the by Operator graph...


In [ ]:
# Kyle - Word cloud of Disengagement descriptions
text = combined_dis_df["DESCRIPTION OF FACTS CAUSING DISENGAGEMENT"].values 

stop_words = ["vehicle", "/n", "Non", "road"] + list(STOPWORDS)
wordcloud = WordCloud(stopwords = stop_words, background_color="white", colormap="CMRmap").generate(str(text))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()



In [ ]:
### Miles driven donut by operator? only a few DOMINATE the space.
## No need? Show by above graphs?

In [ ]:
### Check if any manufacturers have multiple permit numbers?

In [ ]:
# sunburst of mileage by month? 
# what sotry? weather vehicles can/can't run in


In [ ]:
### Scatter of number of VINs and total miles? by manufacturer
# Bubble size number of disengagements?

In [ ]:
### Scatter of x=miles driven, y=disengagements, by VIN? color by manufacturer

In [ ]:
#### Total Miles driven per year ####
# Aggregate the sum of "Mileage Total" by "Year" and "Manufacturer"
agg_df = mileage_df.groupby(['Manufacturer', 'Year'])['Mileage Total'].sum().reset_index()

# Filter to include only data for the top 10 operators
top_10_operators = mileage_df.groupby('Manufacturer')['Mileage Total'].sum().nlargest(10).index
top_miles = agg_df[agg_df['Manufacturer'].isin(top_10_operators)]
manufacturer_order = agg_df.groupby('Manufacturer')['Mileage Total'].sum().sort_values(ascending=False).index


# Create the bar plot
fig = px.bar(top_miles, x='Year', y='Mileage Total', color='Manufacturer', text="Manufacturer", title='Annual Autonomous Mileage Totals', category_orders={'Manufacturer': manufacturer_order})
fig.update_layout(xaxis_title='Year', yaxis_title='Mileage Total')
fig.update_xaxes(tickvals=[2021, 2022, 2023])
fig.show()


In [ ]:
### Duplicate of above, but line

# Create line plot
fig = px.line(top_miles, x='Year', y='Mileage Total', color='Manufacturer', text="Manufacturer", title='Annual Mileage Total by Operator')
fig.update_layout(xaxis_title='Year', yaxis_title='Mileage Total')
fig.update_xaxes(tickvals=[2021, 2022, 2023])
fig.show()

In [ ]:
#### DISENGAGEMENTS PER THOUSAND MILES DRIVEN
# Makes most sense to show by year on a per-company basis. 

# Group by manufacturer - get VIN count, total disengagments and total miles
dptm = nonzero_mileage.groupby(['Manufacturer', 'Year']).agg({'Mileage Total': 'sum',
                                                              'Annual Total of Disengagements': 'sum',
                                                              'VIN NUMBER': 'nunique'}).reset_index()
# Calculate DPTM
dptm["Disengagements/Thousand Miles"] = (dptm['Annual Total of Disengagements'] / (dptm['Mileage Total'] / 1000))

# Lowest DPTM Operators
smoothest_operators = dptm.groupby('Manufacturer')['Disengagements/Thousand Miles'].sum().nsmallest(10).index
# Simple Yearly Aggregate of DPTM per Operator
agg_2 = dptm.groupby(['Manufacturer', 'Year'])['Disengagements/Thousand Miles'].sum().reset_index()
# agg_2 filtered by smoothest operators (best 10)
lowest_disengage_rate = agg_2[agg_2['Manufacturer'].isin(smoothest_operators)]

# DPTM df filtered down to 10 best @ DPTM
dptm_filtered_10 = dptm[dptm['Manufacturer'].isin(smoothest_operators)]

dptm_total = nonzero_mileage.groupby(['Manufacturer']).agg({'Mileage Total': 'sum',
                                                              'Annual Total of Disengagements': 'sum'}).reset_index()
dptm_total["Disengagements/Thousand Miles"] = (dptm_total['Annual Total of Disengagements'] / (dptm_total['Mileage Total'] / 1000))

dptm_total.head()

In [ ]:
### Do the correlation thing on DPTM:

In [ ]:
### Graphing DPTM
# Mehhhhhhhhhhhhhhhhhhhhhhhhhhhhhhh tells no story, would be a lot cooler with more years of data
fig = px.scatter(dptm, x="Mileage Total", y="Annual Total of Disengagements", 
                 size="VIN NUMBER", color="Manufacturer",
                 animation_frame="Year", animation_group="Manufacturer",
                 range_x=[0, dptm["Mileage Total"].max() * 1.1],  # Adjust range for x-axis
                 range_y=[0, dptm["Annual Total of Disengagements"].max() * 1.1],  # Adjust range for y-axis
                 labels={"Mileage Total": "Annual Mileage", "Annual Total of Disengagements": "Annual Total of Disengagements", "VIN NUMBER": "Number of Vehicles", "Manufacturer": "Manufacturer", "Year": "Year"}
                )
fig.update_layout(title='Total Disengagments vs Total Mileage [Bubble size = No. of Vehicles]')
fig.show()

In [ ]:
### Graphing RATE of dsengagements against total miles
fig = px.scatter(dptm, x="Mileage Total", y="Disengagements/Thousand Miles", color="Manufacturer")
fig.update_layout(title='Average Annual Disengagements Per Thousand Miles v. Total Mileage - All Opertors')
fig.show()

fig = px.scatter(dptm_total, x="Mileage Total", y="Disengagements/Thousand Miles", color="Manufacturer")
fig.update_layout(title='Average Disengagements Per Thousand Miles v. Total Mileage - All Opertors (2021-2023)')

fig.show()


In [ ]:
# INFO: mileage_df.shape = (4360, 46)
print(mileage_df.columns)
mileage_df.head()


In [ ]:
# INFO: disengagement_df.shape = (23579, 10)
print(disengagement_df.columns)